In [3]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('/cellar/users/snwright/Git/Network_Evaluation_Tools/neteval/')
from node_annotation import *

In [6]:
rc_data = '/cellar/users/snwright/Data/RareCommon/'

### Combine results

In [75]:
file_list = ['mrna_annotations.txt', 'cite_annotations.txt', 'conservation_annotations.txt', 'prot_annotations.txt']
dfs = []
for f in file_list:
    dfs.append(pd.read_csv(os.path.join(rc_data, 'inputs', f), sep='\t', index_col=0))
all_results = pd.concat(dfs, axis=1).sort_index()

In [77]:
all_results.to_csv(os.path.join(rc_data, 'inputs', 'combined_annotations.Nov26_24.txt'), sep='\t')

In [78]:
os.path.join(rc_data, 'inputs', 'combined_annotations.Nov26_24.txt')

'/cellar/users/snwright/Data/RareCommon/inputs/combined_annotations.Nov26_24.txt'

### Create Symbol-Entrez Dictionary?

In [30]:
hgnc = pd.read_csv(os.path.join(rc_data, 'Reference', 'HGNC_download_Dec20_2023.txt'), sep='\t').dropna(subset='NCBI Gene ID')
hgnc['NCBI Gene ID'] = hgnc['NCBI Gene ID'].astype(int)

In [34]:
hgnc_to_entrez = hgnc.set_index('Approved symbol').loc[:, ('NCBI Gene ID')].to_dict()

### mRNA

In [7]:
mrna = ExpressionData(os.path.join(rc_data, 'Reference', 'gtex_median_processed_1.tsv.gz'))

In [40]:
mrna.data['Entrez'] = mrna.data['Entrez'].astype(int)

In [42]:
mean_mrna = mrna.data.set_index('Entrez', drop=True).iloc[:, 2:].mean(axis=1)
median_mrna = mrna.data.set_index('Entrez', drop=True).iloc[:, 2:].median(axis=1)
non_zero_mrna = (mrna.data.set_index('Entrez', drop=True).iloc[:, 2:] > 0).sum(axis=1)

In [50]:
mrna_data = pd.DataFrame({'mean_rna': mean_mrna, 'median_rna': median_mrna, 'non_zero_mrna': non_zero_mrna})
mrna_data.index.name= None

In [51]:
mrna_data.to_csv(os.path.join(rc_data, 'inputs', 'mrna_annotations.txt'), sep='\t')

In [52]:
mrna_data.head()

,mean_rna,median_rna,non_zero_mrna
102466751,0.000000,0.000000,0
645520,0.000000,0.000000,0
79504,0.020226,0.022315,29
403263,0.038865,0.042189,43
79501,0.048267,0.045494,47


### Protein

Done in neteval because for some reason this is not working

### Citation

In [57]:
citation_counts = pd.read_csv(os.path.join(rc_data, 'Reference', 'gene_citation_counts_Dec20_2023.txt'), sep='\t', header=None, names=['GeneID', 'CitationCount'], index_col=0)

In [59]:
citation_counts.index.name=None

In [60]:
citation_counts.to_csv(os.path.join(rc_data, 'inputs', 'cite_annotations.txt'), sep='\t')

### Conservation

In [13]:
cons_scores = pd.read_csv(os.path.join(rc_data, 'Reference', 'gene_conservation_scores.txt'), sep='\t', index_col=0)

In [63]:
cons_scores = cons_scores.set_index('entrezgene', drop=True)
cons_scores.index.name = None

In [66]:
cons_scores = cons_scores.rename(columns={'mean':'mean_phylop', 'median':'median_phylop'})

In [68]:
cons_scores.loc[:, ('mean_phylop', 'median_phylop')].to_csv(os.path.join(rc_data, 'inputs', 'conservation_annotations.txt'), sep='\t')

### GO annotations

### Disgen Associations

### Drug targets

### Monogenic disorder genes